In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import pymysql
from sqlalchemy import create_engine
import json
import warnings
import nltk
from nltk.corpus import stopwords
from scipy.sparse import hstack
warnings.filterwarnings('ignore')

In [ ]:
# параметры подключения к базе данных
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'ame0372',
    'database': 'froglib'
}

# создание подключения
connection = pymysql.connect(**db_config)

In [ ]:
# загрузка данных из БД
def load_data_from_db():
    books_query = """
    SELECT b.id_book, b.title_book, b.description_book, b.id_category, c.name_category 
    FROM Books b
    JOIN Categories c ON b.id_category = c.id_category
    """
    books = pd.read_sql(books_query, connection)

    user_books_query = """
    SELECT id_user, id_book, id_list_type 
    FROM UserBooks
    """
    user_books = pd.read_sql(user_books_query, connection)

    ratings_query = """
    SELECT id_user, id_book, rating 
    FROM BookRatings
    """
    ratings = pd.read_sql(ratings_query, connection)

    views_query = """
    SELECT id_user, id_entity as id_book 
    FROM ViewHistory
    WHERE type_entity = 'Книга'
    """
    views = pd.read_sql(views_query, connection)

    return books, user_books, ratings, views

In [4]:
books, user_books, ratings, views = load_data_from_db()

In [ ]:
# предобработка данных
def preprocess_data(books, user_books, ratings, views):
    # создание весов для списков
    list_weights = {
        1: 3,
        2: 2,
        3: -1,
        4: 4,
        5: 5
    }

    # добавление веса к спискам пользователей
    user_books['weight'] = user_books['id_list_type'].map(list_weights)

    # объединение оценки и списка
    user_interactions = pd.concat([
        ratings.assign(weight=ratings['rating'] * 2),
        user_books[['id_user', 'id_book', 'weight']]
    ]).drop_duplicates(['id_user', 'id_book'], keep='first')

    # добавление просмотров с меньшим весом
    if not views.empty:
        views['weight'] = 1
        user_interactions = pd.concat([
            user_interactions,
            views[['id_user', 'id_book', 'weight']]
        ]).drop_duplicates(['id_user', 'id_book'], keep='first')

    # очистка описаний книг
    books['clean_description'] = books['description_book'].fillna('').str.lower()

    return books, user_interactions

In [6]:
books, user_interactions = preprocess_data(books, user_books, ratings, views)

In [ ]:
# загрузка русский стоп-слов
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Я\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# контентная фильтрация
def content_based_recommendation(books):
    tfidf = TfidfVectorizer(stop_words=russian_stopwords, max_features=5000)
    tfidf_matrix = tfidf.fit_transform(books['clean_description'])

    # добавление категорий как доп признаки
    category_dummies = pd.get_dummies(books['name_category'], sparse=True)

    # объединение признаков
    content_features = csr_matrix(hstack([tfidf_matrix, category_dummies]))

    # вычисление косинусного сходства
    content_similarity = cosine_similarity(content_features)

    return content_similarity

In [10]:
content_similarity = content_based_recommendation(books)

In [ ]:
# коллаборативная фильтрация
def collaborative_filtering(user_interactions, books):
    # матрица пользователь-книга
    user_books_matrix = user_interactions.pivot_table(
        index='id_user',
        columns='id_book',
        values='weight',
        fill_value=0
    )

    # нормализация оценок
    user_means = user_books_matrix.mean(axis=1)
    user_books_matrix = user_books_matrix.sub(user_means, axis=0)

    n_features = user_books_matrix.shape[1]
    n_components = min(50, n_features)  # ограничение числа компонентов
    
    # SVD для уменьшения размерности
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    user_factors = svd.fit_transform(user_books_matrix)
    book_factors = svd.components_.T

    # вычисление предсказанных оценок
    predicted_ratings = np.dot(user_factors, book_factors.T)

    return predicted_ratings, user_books_matrix.index, user_books_matrix.columns

In [ ]:
predicted_ratings, user_ids, book_ids = collaborative_filtering(user_interactions, books)

In [ ]:
# гибридные рекомендации
def hybrid_recommendations(user_id, content_weight=0.6, collab_weight=0.4):
    try:
        # получение индекса пользователя для коллаборативной фильтрации
        user_idx = np.where(user_ids == user_id)[0][0]
        # получение предсказанных оценок от коллаборативной фильтрации
        collab_scores = predicted_ratings[user_idx]
        
        # получение книг, с которыми пользователь уже взаимодействовал
        interacted_books = user_interactions[user_interactions['id_user'] == user_id]['id_book'].values
        
        # усреднение сходства по любимым книгам для контентной фильтрации
        favorite_books = user_interactions[
            (user_interactions['id_user'] == user_id) &
            (user_interactions['weight'] >= 4) # Любимые и прочитанные
        ]['id_book'].values

        if len(favorite_books) > 0:
            # нахождение индексов любимых книг
            favorite_indices = books[books['id_book'].isin(favorite_books)].index
            # усреднение сходства с любимыми книгами
            content_scores = content_similarity[favorite_indices].mean(axis=0)
        else:
            # если нет любимых, используются все взаимодействия
            interacted_indices = books[books['id_book'].isin(interacted_books)].index
            if len(interacted_indices) > 0:
                content_scores = content_similarity[interacted_indices].mean(axis=0)
            else:
                # если нет взаимодействий, возвращение популярных
                popular_books = user_interactions['id_book'].value_counts().head(5).index
                result = books[books['id_book'].isin(popular_books)][['id_book', 'title_book', 'name_category']]
                result['hybrid_score'] = 1.0
                return result.to_dict('records')
        
        # нормализация оценок
        content_scores = (content_scores - content_scores.min()) / (content_scores.max() - content_scores.min())
        collab_scores = (collab_scores - collab_scores.min()) / (collab_scores.max() - collab_scores.min())

        # формирование рекомендаций
        recommendations = pd.DataFrame({
            'id_book': book_ids,
            'content_score': content_scores,
            'collab_score': collab_scores
        })

        # расчёт гибридной оценки
        recommendations['hybrid_score'] = (
            recommendations['content_score'] * content_weight +
            recommendations['collab_score'] * collab_weight
        )

        # исключение уже взаимодействовавших книги
        recommendations = recommendations[~recommendations['id_book'].isin(interacted_books)]

        # сортировка топ-5
        recommendations = recommendations.sort_values('hybrid_score', ascending=False).head(5)

        # добавление информации о книгах
        recommendations = recommendations.merge(
            books[['id_book', 'title_book', 'name_category']],
            on='id_book',
            how='left'
        )

        return recommendations[['id_book', 'title_book', 'name_category', 'hybrid_score']].to_dict('records')
    
    except IndexError:
        # для новых пользователей популярные книги
        popular_books = user_interactions['id_book'].value_counts().head(5).index
        result = books[books['id_book'].isin(popular_books)][['id_book', 'title_book', 'name_category']]
        result['hybrid_score'] = 1.0
        return result.to_dict('records')

In [14]:
sample_user_id = 6
recommendations = hybrid_recommendations(sample_user_id)
print('Рекомендации для пользователя:')
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec['title_book']} ({rec['name_category']}) - score: {rec['hybrid_score']:.2f}")

Рекомендации для пользователя:
1. Мастер и Маргарита (Художественная литература) - score: 0.57
2. Атлант расправил плечи (Художественная литература) - score: 0.57
3. Богатый папа, бедный папа (Бизнес и экономика) - score: 0.39
4. Город женщин (Художественная литература) - score: 0.15
5. Война и мир (Художественная литература) - score: 0.04
